In [3]:
import pandas as pd
import numpy as np
import re
import time
import pickle as pkl
import seaborn as sns; sns.set(style="ticks", color_codes=True)
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

from annoy import AnnoyIndex
from sklearn.decomposition import TruncatedSVD

In [18]:
df_for_brands = pkl.load(open('df_pickles/df_reg_1.p','rb'))
df_for_brands.drop('price',1,inplace=True)
df_for_brands_gas = df_for_brands[df_for_brands['Engine type']!='Electric']
df_for_brands_elec = df_for_brands[df_for_brands['Engine type']=='Electric']
prices = pkl.load(open('df_pickles/prices.p','rb'))
df_for_brands_gas = pd.concat([df_for_brands_gas, prices],1)
df_gas = pkl.load(open('df_pickles/df_gas_sc_dmd.p','rb'))
df_gas_mod = df_gas.drop(['model', 'front_suspension_type_1',
       'front_suspension_type_2', 'front_suspension_type_3',
       'front_suspension_type_4', 'front_suspension_type_5',
       'front_suspension_type_6', 'front_suspension_type_7',
       'front_suspension_type_8', 'front_suspension_type_9', 
        'front_suspension_type_10', 'front_suspension_type_11',
       'front_suspension_type_12', 'front_wheel_diameter_15',
       'front_wheel_diameter_16', 'front_wheel_diameter_17',
       'front_wheel_diameter_18', 'front_wheel_diameter_19',
       'front_wheel_diameter_20', 'front_wheel_diameter_21',
       'front_wheel_diameter_22', 'rear_suspension_type_1',
       'rear_suspension_type_2', 'rear_suspension_type_3',
       'rear_suspension_type_4', 'rear_suspension_type_5', 
                          'rear_suspension_type_6',
       'rear_suspension_type_7', 'rear_suspension_type_8',
       'rear_suspension_type_9', 'rear_suspension_type_10',
       'rear_wheel_diameter_15', 'rear_wheel_diameter_16',
       'rear_wheel_diameter_17', 'rear_wheel_diameter_18',
       'rear_wheel_diameter_19', 'rear_wheel_diameter_20',
       'rear_wheel_diameter_21', 'rear_wheel_diameter_22',
                         'brand_1',
       'brand_2', 'brand_3', 'brand_4', 'brand_5', 'brand_6', 'brand_7',
       'brand_8', 'brand_9', 'brand_10', 'brand_11', 'brand_12', 'brand_13',
       'brand_14', 'brand_15', 'brand_16', 'brand_17', 'brand_18', 'brand_19',
       'brand_20', 'brand_21', 'brand_22', 'brand_23', 'brand_24', 'brand_25',
       'brand_26', 'brand_27', 'brand_28', 'brand_29', 'brand_30', 'brand_31',
       'brand_32', 'brand_33', 'brand_34', 'brand_35', 'brand_36', 'brand_37',
       'brand_39', 'brand_40', 'brand_41'], 1)
df_gas_mod['index'] = list(df_gas_mod.index)
df_for_brands_gas['index'] = list(df_for_brands_gas.index)

In [7]:
def threed_figures(model, df):
    fignum = 1
    fig = plt.figure(fignum, figsize=(20, 10))
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
    labels = model.labels_

    ax.scatter(df['x'], df['y'], df['z'],
               c=df['labels'].astype(np.float), edgecolor='k')

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel('PCA1')
    ax.set_ylabel('PCA2')
    ax.set_zlabel('PCA3')
    ax.set_title(str(model))
    ax.dist = 12

    # Plot the ground truth
    fig = plt.figure(fignum, figsize=(20, 10))
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

#     for name, label in [('PCA1', 0), ('PCA2', 1), ('PCA3', 2)]:
#         ax.text3D(df[y == label, 3].mean(),
#                   df[y == label, 0].mean(),
#                   df[y == label, 2].mean() + 2, name,
#                   horizontalalignment='center',
#                   bbox=dict(alpha=.2, edgecolor='w', facecolor='w'))
    #Reorder the labels to have colors matching the cluster results
#    y = np.choose(df['labels'], [1, 2, 0]).astype(np.float)
    ax.scatter(df['x'], df['y'], df['z'],
               c=df['labels'], edgecolor='k',cmap='hsv',s=16)

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel('PCA1')
    ax.set_ylabel('PCA2')
    ax.set_zlabel('PCA3')
    ax.set_title('Car Cluster Representation After Reducing Dimension to 3D')
    ax.dist = 12

    fig.show();

In [67]:
def annoy():    
    svd = TruncatedSVD(n_components=11)
    df_Annoy_svd = svd.fit_transform(df_gas_mod.iloc[:,:-1])
    from annoy import AnnoyIndex
    import random

    f = df_Annoy_svd.shape[1] # Length of item vector that will be indexed
    t = AnnoyIndex(f)
    for i in range(df_Annoy_svd.shape[0]):
        v = df_Annoy_svd[i]
        t.add_item(i, v)

    t.build(15)
    t.save('test.ann')

def nearest_car_Annoy(df, car_idx, index, n, print_output=True):
    nn = index.get_nns_by_item(car_idx, n)
    if print_output == True:
        print('Closest to %s : \n' % df.index[car_idx])
    cars = [df.index[i] for i in nn]
    if print_output == True:
        return df_for_brands_gas.loc[cars, ['brand','model','Torque','Passenger Capacity', 'price', 'trim']]

def index_finder(string1,string2):
    index_list = []
    for i in df_for_brands_gas.index:
        if re.findall(str(string1).lower()+'\/'+str(string2).lower(), i):
            index_list.append(i)
    return df_for_brands_gas.loc[index_list, ['brand','model','Torque','Passenger Capacity', 'price', 'trim']]

In [ ]:
# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# from dash.dependencies import Input, Output, State

# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# app.layout = html.Div([
#     dcc.Markdown('Car'),
#     dcc.Input(id='input-1-state', type='text', value=''),
#     dcc.Markdown('Model'),
#     dcc.Input(id='input-2-state', type='text', value=''),
#     html.Button(id='submit-button', n_clicks=0, children='Submit'),
#     html.Div(id='output-state')
# ])


# def generate_table(dataframe, max_rows=26):
#     return html.Table(
#         # Header
#         [html.Tr([html.Th(col) for col in dataframe.columns]) ] +
#         # Body
#         [html.Tr([
#             html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
#         ]) for i in range(min(len(dataframe), max_rows))]
#     )

# @app.callback(Output('output-state', 'children'),
#               [Input('submit-button', 'n_clicks')],
#               [State('input-1-state', 'value'),
#                State('input-2-state', 'value')])
# def update_output(n_clicks, input1, input2):
#     index_list = []
#     for i in df_for_brands_gas.index:
#         if re.findall(str(input1).lower()+'\/'+str(input2).lower(), i):
#             index_list.append(i)
#     df = df_for_brands_gas.loc[index_list, ['brand','model','Torque','Passenger Capacity', 'price', 'trim']]
#     return generate_table(df, max_rows=26)

In [64]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Car Recommender'),
    
    dcc.Markdown('''Enter your favorite car brand and model to get other cars with similar specs.'''),
    
    dcc.Markdown('Car brand'),
    dcc.Input(id='input-1-state', type='text', value=''),
    
    dcc.Markdown('Model'),
    
    dcc.Input(id='input-2-state', type='text', value=''),
    
    html.Button(id='submit-1-button', n_clicks=0, children='Submit'),
    html.Div(id='output-1-state'),
    dcc.Markdown('Paste Index Here'),
    dcc.Input(id='input-3-state', type='text', value=''),
    html.Button(id='submit-2-button', n_clicks=0, children='Submit'),
    html.Div(id='output-2-state')
])

def generate_table(dataframe, max_rows=26):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns]) ] +
        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )

svd = TruncatedSVD(n_components=11)
df_Annoy_svd = svd.fit_transform(df_gas_mod.iloc[:,:-1])
from annoy import AnnoyIndex
import random

f = df_Annoy_svd.shape[1] # Length of item vector that will be indexed
t = AnnoyIndex(f)  
for i in range(df_Annoy_svd.shape[0]):
    v = df_Annoy_svd[i]
    t.add_item(i, v)

t.build(15)
t.save('test.ann')

@app.callback(Output('output-1-state', 'children'),
              [Input('submit-1-button', 'n_clicks')],
              [State('input-1-state', 'value'),
               State('input-2-state', 'value')])
def update_output1(n_clicks, input1, input2):
    print_output1 = True
    index_list = []
    for i in df_for_brands_gas.index:
        if re.findall(str(input1).lower()+'\/'+str(input2).lower(), i):
            index_list.append(i)
    if print_output1 == True:
        df = df_for_brands_gas.loc[index_list, ['brand','model','Torque','Passenger Capacity', 'price', 'trim', 'index']]
    return generate_table(df, max_rows=26)



@app.callback(Output('output-2-state', 'children'),
              [Input('submit-2-button', 'n_clicks')],
              [State('input-3-state', 'value')])
def update_output2(n_clicks, input3):
    n=10
    print_output=True
    index = t
    val3_iloc = list(df_gas_mod.index).index(input3)
    nn = index.get_nns_by_item(val3_iloc, n)
    if print_output == True:
        print('Closest to %s : \n' % df_gas_mod.index[val3_iloc])
    cars = [df_gas_mod.index[i] for i in nn]
    if print_output == True:
        df = df_for_brands_gas.loc[cars, ['brand','model','Torque','Passenger Capacity', 'price', 'trim', 'index']]
    return generate_table(df, max_rows=10)

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: FutureWarning:

The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.



In [65]:
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Oct/2019 23:41:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2019 23:41:38] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.2&m=1571606833 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2019 23:41:38] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.2&m=1571606833 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2019 23:41:38] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.2&m=1571606833 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2019 23:41:38] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.1&m=1571606836 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2019 23:41:38] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.3.1&m=1571606834 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2019 23:41:38] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.1.2&m=1571606833 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2019 2